# Bergamini Federico 845646

Inizializzo i parametri e importo le librerie, in particolare:
- il nome del file da validare. 
- il nome del file contenente il report delle violazioni.
- importo sys e re
-dichiaro due espressioni regolari utili per il gene_id e il transcript_id

In [90]:
import sys
import re
gtf_file_name = './error_files/err7.gtf' #per testare gli errori, inserire il nome del file che si vuole testare
#gtf_file_name = './input.gtf'
report_file_name = './report.txt'
gene_regexpr = 'gene_id\s+"([^"]+)";'
transcript_regexpr = 'transcript_id\s+"([^"]+)";'

**Funzione write_error:**

**@parameter** *report_file_name* `Nome del file di report`

**@parameter** *error_comment* `Commento di un errore commesso nel file GTF`

**@parameter** *error_line* `Linea a cui è stato commesso l'errore`

**@post_condition** `Viene aggiunto un record nel file di report contenente l'ettore passato come parametro` 

In [91]:
def write_error(report_file_name, error_comment, error_line):
    with open(report_file_name, 'a') as report_output_file:
        report_output_file.write('Error at line ' + error_line + '\n\t' + error_comment + '\n\n')

**Funzione write_report:**

**@parameter** *report_file_name* `Nome del file di report`

**@parameter** *comment* `Stringa da scrivere sul file di report`

**@post_condition** `La stringa viene scritta sul file di report`

In [92]:
def write_report(report_file_name, comment):
    with open(report_file_name, 'a') as report_output_file:
        report_output_file.write(comment + '\n\n')

**Creo il file report.txt, nel caso fosse già presente ne elimino il contenuto**

In [93]:
with open(report_file_name, 'w') as report_output_file:
        report_output_file.write('Inizio della validazione\n\n')

**Leggo il file in formato GTF eliminando i commenti.**

In [94]:
with open(gtf_file_name, 'r') as gtf_input_file:
    gtf_file_rows = gtf_input_file.readlines() #leggo i record

for row in gtf_file_rows:
    if '#' in row: #elimino i commenti
        gtf_file_rows[gtf_file_rows.index(row)] = gtf_file_rows[gtf_file_rows.index(row)][:row.index('#')]

gtf_file_rows = [row for row in gtf_file_rows if row.rstrip() != ''] #elimino i record di solo commento

In [95]:
gtf_file_rows

['140\tTwinscan\t3UTR\t65149\t65487\t.\t-\t.\tgene_id "140.000"; transcript_id "140.000.1"; \n',
 '140\tTwinscan\t3UTR\t66823\t66992\t.\t-\t.\tgene_id "140.000"; transcript_id "140.000.1";\n',
 '140\tTwinscan\tstop_codon\t66993\t66995\t.\t-\t0\tgene_id "140.000"; transcript_id "140.000.1";\n',
 '140\tTwinscan\tCDS\t66996\t66999\t.\t-\t1\tgene_id "140.000"; transcript_id "140.000.1";\n',
 '140\tTwinscan\tintron_CNS\t70103\t70151\t*\t-\t.\tgene_id "140.000"; transcript_id "140.000.1";  ',
 '140\tTwinscan\tCDS\t70207\t70294\t.\t-\t2\tgene_id "140.000"; transcript_id "140.000.1";\n',
 '140\tTwinscan\tCDS\t71696\t71807\t.\t-\t0\tgene_id "140.000"; transcript_id "140.000.1";\n',
 '140\tTwinscan\tstart_codon\t71805\t71806\t.\t-\t0\tgene_id "140.000"; transcript_id "140.000.1";\n',
 '140\tTwinscan\tstart_codon\t73222\t73222\t.\t-\t2\tgene_id "140.000"; transcript_id "140.000.1";\n',
 '140\tTwinscan\tCDS\t73222\t73222\t.\t-\t0\tgene_id "140.000"; transcript_id "140.000.1";\n',
 '140\tTwinscan\t

## Controllo validità del file GTF

**Controllo che il numero di campi, separati da tabulazione, per ogni record del file, sia uguale a 9.** 

In caso contrario stampo l'errore e fermo l'esecuzione.

In [96]:
for row in gtf_file_rows:
    n_fields = len(row.rstrip().split('\t'))
    if n_fields != 9:
        n_row = gtf_file_rows.index(row)
        error_comment = 'Il numero dei campi è ' + str(n_fields) + ', dovrebbe essere 9'
        write_error(report_file_name, error_comment, str(n_row))
        sys.exit()

write_report(report_file_name, 'Il numero dei campi è rispettato per ogni record')

**Controllo che il campo `<source>` sia univoco all'interno del file.**

In caso contrario stampo l'errore e fermo l'esecuzione.

In [97]:
for row in gtf_file_rows[1:]:
    curr_source = row.rstrip().split('\t')[1]
    prec_source_index = gtf_file_rows.index(row) - 1
    prec_source = gtf_file_rows[prec_source_index].rstrip().split('\t')[1]
    if prec_source != curr_source: #se il campo source corrente è diverso dal precedente ho un errore
        error_comment = 'Il campo <source> assume valore \'' + curr_source + '\' che è diverso dal valore del record precedente'
        n_row = prec_source_index + 1;
        write_error(report_file_name, error_comment, str(n_row))
        sys.exit()

write_report(report_file_name, 'Il campo <source> è univoco in tutto il file')

**Controllo che il campo `feature` sia ammissibile per il formato GTF**

In caso contrario stampo l'errore e fermo l'esecuzione.

Inizializzo la lista dei possibili valori per il campo feature

In [98]:
possible_feature_values = ['5UTR', '3UTR', 'start_codon', 'stop_codon', 'CDS', 'inter', 'inter_cns', 'intron_CNS', 'exon']

Per ogni record controllo che il valore di `feature` sia in `possible_feature_values`, altrimenti stampo l'errore e fermo l'esecuzione.

In [99]:
for row in gtf_file_rows:
    feature = row.rstrip().split('\t')[2]
    if feature not in possible_feature_values: #ho un errore
        error_comment = 'Il campo <feature> assume valore \'' + feature + '\' che non è incluso in ' + str(possible_feature_values)
        n_row = gtf_file_rows.index(row)
        write_error(report_file_name, error_comment, str(n_row))
        sys.exit()
        
write_report(report_file_name, 'Il campo <feature> assume valori ammissibili per ogni record del file')

**Controllo la presenza di `gene_id` e `transcript_id` nel campo `[attributes]`.** 

Nel caso uno dei due non fosse presente stampo l'errore e fermo l'esecuzione.

In [100]:
for row in gtf_file_rows:
    if len(re.findall(gene_regexpr, row.rstrip().split('\t')[8])) != 1:
        error_comment = 'Il campo <attributes> non contiene l\'attributo gene_id'
        n_row = gtf_file_rows.index(row)
        write_error(report_file_name, error_comment, str(n_row))
        sys.exit()
    if len(re.findall(transcript_regexpr, row.rstrip().split('\t')[8])) != 1:
        error_comment = 'Il campo <attributes> non contiene l\'attributo trancript_id'
        n_row = gtf_file_rows.index(row)
        write_error(report_file_name, error_comment, str(n_row))
        sys.exit()

write_report(report_file_name, 'Il campo <attributes> contiene gene_id e transcript_id per ogni record del file')

**Controllo che il campo `<seqname>` sia univoco per ogni gene nel file.**

In [101]:
seqname_dict = dict() #dizionario che associa ad ogni gene il proprio set di <seqname>

for row in gtf_file_rows: #costruisco seqname_dict
    seqname = row.rstrip().split('\t')[0]
    hugo_name = re.findall(gene_regexpr, row.rstrip().split('\t')[8])[0] 
    
    seqname_set = seqname_dict.get(hugo_name, set())
    seqname_set.add(seqname)
    seqname_dict.update([(hugo_name, seqname_set)])

for gene in seqname_dict:
    if len(seqname_dict[gene]) != 1: #in caso uno dei set associati ai geni abbia cardinalità diversa da 1 ho un errore 
        wrong_seqname_rows = [row for row in gtf_file_rows if re.findall(gene_regexpr, row.rstrip().split('\t')[8])[0] == gene]
        for row in wrong_seqname_rows[1:]: #rinraccio l'errore
            curr_seqname = row.rstrip().split('\t')[0]
            prec_seqname_index = wrong_seqname_rows.index(row) - 1
            prec_seqname = wrong_seqname_rows[prec_seqname_index].rstrip().split('\t')[0]
            if curr_seqname != prec_seqname: #ho un errore alla linea corrente
                error_comment = 'Il campo <seqname> assume valore \'' + curr_seqname + '\' che è diverso dal valore del record precedente'
                n_row = gtf_file_rows.index(row) #reperisco il record in gtf_file_rows
                write_error(report_file_name, error_comment, str(n_row))
                sys.exit()

write_report(report_file_name, 'Il campo <seqname> è univoco per ogni gene contenuto nel file')

**Controllo che i campi `<start>` e `<end>` siano ammissibili:**

- controllo che siano numeri
- controllo che siano non negativi
- controllo che siano `<end>` >= `<start>`

In caso contrario stampo l'errore e fermo l'esecuzione.

In [102]:
for row in gtf_file_rows:
    start = row.rstrip().split('\t')[3]
    end = row.rstrip().split('\t')[4]
    #controllo che siano interi >= 0
    if start.isdigit() == False:
        error_comment = 'Il campo <start> assume il valore ' + start + '. Non è un intero'
        n_row = gtf_file_rows.index(row)
        write_error(report_file_name, error_comment, str(n_row))
        sys.exit()
    if end.isdigit() == False:
        error_comment = 'Il campo <end> assume il valore ' + end + '. Non è un intero'
        n_row = gtf_file_rows.index(row)
        write_error(report_file_name, error_comment, str(n_row))
        sys.exit()
    start = int(start)
    end = int(end)
    length = end - start
    if end < 1 : #controllo sia diverso da 0
        error_comment = 'Il campo <end> assume il valore : ' + end + '. Non è un intero positivo'
        n_row = gtf_file_rows.index(row)
        write_error(report_file_name, error_comment, str(n_row))
        sys.exit()
    if start < 1: #controllo che sia diverso da 0
        error_comment = 'Il campo <start> assume il valore : ' + start + '. Non è un intero positivo'
        n_row = gtf_file_rows.index(row)
        write_error(report_file_name, error_comment, str(n_row))
        sys.exit()
    if length < 0: #controllo che length sia ammissibile
        error_comment = '<end> - <start> assume il valore ' + str(length) + '. E\' minore di 0'
        n_row = gtf_file_rows.index(row)
        write_error(report_file_name, error_comment, str(n_row))
        sys.exit()

write_report(report_file_name, 'I campi <end> e <start> sono ammissibili per ogni record del file')

**Controllo la validità del campo `<score>`:**

- controllo che sia un intero
- controllo che sia un float
- controllo che sia un punto

In caso contrario stampo l'errore e fermo l'esecuzione.

In [103]:
for row in gtf_file_rows:
    score = row.rstrip().split('\t')[5]
    if len(re.findall("(\d+(\.\d+)*)|(\.)", score)) != 1 : #controllo che sia un intero, un float o un '.'
        error_comment = 'Il campo <score> non è nè un punto, nè un numero decimale (separato da punto), nè un numero intero'
        n_row = gtf_file_rows.index(row)
        write_error(report_file_name, error_comment, str(n_row))
        sys.exit()

write_report(report_file_name, 'Il campo <score> è corretto per ogni record del file')

[('', '', '.')]
[('', '', '.')]
[('', '', '.')]
[('', '', '.')]
[]


SystemExit: 

**Controllo che il campo `<strand>` assuma solo valori '-' o '+'**

In caso contrario stampo l'errore e fermo l'esecuzione.

In [205]:
for row in gtf_file_rows:
    strand = row.rstrip().split('\t')[6]
    if strand not in ('+', '-'):
        error_comment = 'Il campo <strand> assume il valore ' + strand + '. Il valore del campo <strand> può assumere i valori \'+\' o \'-\''
        n_row = gtf_file_rows.index(row)
        write_error(report_file_name, error_comment, str(n_row))
        sys.exit()

write_report(report_file_name, 'il campo <strand> assume un valore ammissibile per ogni record del file')

**Controllo che per ogni gene il campo `<strand>` sia uguale per ogni gene**

In caso contrario stampo l'errore e fermo l'esecuzione.

In [206]:
strand_dict = dict() #dizionario che per ogni gene incluso nel file GTF contiene il set di strand associati

for row in gtf_file_rows: #costruisco il dizionario
    hugo_name = re.findall(gene_regexpr, row.rstrip().split('\t')[8])[0]
    strand = row.rstrip().split('\t')[6]
    
    strand_set = strand_dict.get(hugo_name, set())
    strand_set.add(strand)
    strand_dict.update([(hugo_name, strand_set)])

for hugo_name in strand_dict: 
    strand_set = strand_dict[hugo_name]
    if len(strand_set) != 1: #se il set associato ad ogni gene ha cardinalità diversa da 1 ho una violazione
        wrong_seqname_rows = [row for row in gtf_file_rows if re.findall(gene_regexpr, row.rstrip().split('\t')[8])[0] == hugo_name]
        for row in wrong_seqname_rows[1:]: #rintraccio l'errore
            curr_strand = row.rstrip().split('\t')[6]
            prec_strand_index = wrong_seqname_rows.index(row) - 1
            prec_strand = wrong_seqname_rows[prec_strand_index].rstrip().split('\t')[6]
            if curr_strand != prec_strand: #ho un errore in questo caso
                error_comment = 'Il campo <strand> per il gene ' + hugo_name + ' assume valore diverso da quello del record precedente'
                n_row = gtf_file_rows.index(row)
                write_error(report_file_name, error_comment, str(n_row))
                sys.exit()

write_report(report_file_name, 'il campo <strand> è unico per ogni gene nel file')

**Controllo validità sintattica del campo `<frame>`, deve essere compreso tra 0, 1, 2, '.'**

In caso contrario stampo l'errore e fermo l'esecuzione.

In [207]:
for row in gtf_file_rows:
    frame = row.rstrip().split('\t')[7]
    if frame not in ('0', '1', '2', '.'):
        error_comment = 'Il campo <frame> assume il valore ' + frame + ' dovrebbe essere 0, 1, 2 oppure \'.\''
        n_row = gtf_file_rows.index(row)
        write_error(report_file_name, error_comment, str(n_row))
        sys.exit()

write_report(report_file_name, 'Il campo <frame> assume valori compresi tra 0, 1, 2 e \'.\'')

**Controllo che per la CDS, start_codon e stop_codon il valore del campo `<frame>` sia diverso da '.'**

In caso contrario stampo l'errore e fermo l'esecuzione.

In [208]:
for row in gtf_file_rows:
    frame = row.rstrip().split('\t')[7]
    feature = row.rstrip().split('\t')[2]
    if feature in ('start_codon', 'stop_codon', 'CDS'):
        if frame == '.':
            error_comment = 'Il campo <frame> assume il valore \'.\' per una feature ' + feature + ' dovrebbe essere 0, 1 oppure 2 '
            n_row = gtf_file_rows.index(row)
            write_error(report_file_name, error_comment, str(n_row))
            sys.exit()
    else:
        if frame != '.':
            error_comment = 'Il campo <frame> assume il valore compreso tra ' + str((0, 1, 2)) + ' per una feature ' + feature + ' dovrebbe essere \'.\''
            n_row = gtf_file_rows.index(row)
            write_error(report_file_name, error_comment, str(n_row))
            sys.exit()
     

write_report(report_file_name, 'Il campo <frame> assume valori ammissibili in funzione della feature corrispondente per ogni record')

**Controllo che per ogni trascritto di ogni gene siano predenti le feature `start_codon`, `stop_codon`, `CDS`**

**Al contrario di quanto fornito nelle spefiche, questo controllo non è necessario, ma è stato comunque implementato**

Costruisco un dizionario per ogni feature che faccia corrispondere ad ogni gene i suoi trascritti

id_dict_start = dict() #dizionario per start_codon

for row in gtf_file_rows:
        hugo_name = re.findall(gene_regexpr, row.rstrip().split('\t')[8])[0]
        transcript_id = re.findall(transcript_regexpr, row.rstrip().split('\t')[8])[0]
    
        transcript_dict = id_dict_start.get(hugo_name, dict())
        transcript_dict[transcript_id] = 0
        id_dict_start.update([(hugo_name, transcript_dict)])

import copy #necessario per copiare il dizionario e non il reference al dizionario
#i dizionari per stop_codon e CDS sono identici a quello per start_codon
id_dict_CDS = copy.deepcopy(id_dict_start) 
id_dict_stop = copy.deepcopy(id_dict_start)

Ogni volta che incontro una feature `start_codon`, `stop_codon` o `CDS` incremento il valore del corrispondente trascritto del corrispondente gene del corrispondente dizionario

for row in gtf_file_rows:
    hugo_name = re.findall(gene_regexpr, row.rstrip().split('\t')[8])[0]
    transcript_id = re.findall(transcript_regexpr, row.rstrip().split('\t')[8])[0]
    if row.rstrip().split('\t')[2] == 'start_codon':
        id_dict_start[hugo_name][transcript_id] += 1;
    if row.rstrip().split('\t')[2] == 'stop_codon':
        id_dict_stop[hugo_name][transcript_id] += 1;
    if row.rstrip().split('\t')[2] == 'CDS':
        id_dict_CDS[hugo_name][transcript_id] += 1;

Controllo che per ogni gene, per ogni trascritto, per ogni dizionario, il suo numero di ricorrenze sia > 0

for hugo_name in id_dict_start:
    for transcript in id_dict_start[hugo_name]:
        if id_dict_start[hugo_name][transcript] < 1:
            error_comment = 'Non è presente la feature start_codon per il trascritto ' + transcript + ' del gene ' + hugo_name
            write_error(report_file_name, error_comment, '')
            sys.exit()
for hugo_name in id_dict_stop:
    for transcript in id_dict_stop[hugo_name]:
        if id_dict_stop[hugo_name][transcript] < 1:
            error_comment = 'Non è presente la feature stop_codon per il trascritto ' + transcript + ' del gene ' + hugo_name
            write_error(report_file_name, error_comment, '')
            sys.exit()

for hugo_name in id_dict_CDS:
    for transcript in id_dict_CDS[hugo_name]:
        if id_dict_CDS[hugo_name][transcript] < 1:
            error_comment = 'Non è presente la feature CDS per il trascritto ' + transcript + ' del gene ' + hugo_name
            write_error(report_file_name, error_comment, '')
            sys.exit()

write_report(report_file_name, 'Le feature start_codon, stop_codon, CDS sono presenti per ogni trascritto di ogni gene')

**Per ogni trascritto controllo che gli indici di inizio e di fine degli esoni non si sovappongano** 

In caso contrario stampo l'errore e fermo l'esecuzione.

Costruisco un dizionario che faccia corrispondere ad ogni gene i suoi trascritti

In [209]:
id_dict = dict()

for row in gtf_file_rows:
    if row.rstrip().split('\t')[2] == 'exon':
        hugo_name = re.findall(gene_regexpr, row.rstrip().split('\t')[8])[0]
        transcript_id = re.findall(transcript_regexpr, row.rstrip().split('\t')[8])[0]
    
        transcript_set = id_dict.get(hugo_name, set())
        transcript_set.add(transcript_id)
        id_dict.update([(hugo_name, transcript_set)])

Costruisco un dizionario che faccia corrispondere ad ogni trascritto, una lista di tuple con indice di inizio e fine

In [210]:
transcript_dict = dict()

for row in gtf_file_rows:
    if row.rstrip().split('\t')[2] == 'exon':
        transcript_id = re.findall(transcript_regexpr, row.rstrip().split('\t')[8])[0]
        start = row.rstrip().split('\t')[3]
        end = row.rstrip().split('\t')[4]
    
        start_end_list = transcript_dict.get(transcript_id, list())
        start_end_list.append((start, end))
        transcript_dict.update([(transcript_id, start_end_list)])

Controllo che le tuple non si sovrappongano

In [211]:
for gene in id_dict:
    for transcript in id_dict[gene]:
        transcript_dict[transcript].sort() #ordino le tuple (start, end)
        for start_end in transcript_dict[transcript][:-1]:
            next_index = transcript_dict[transcript].index(start_end) + 1
            next_ele = transcript_dict[transcript][next_index]
            if start_end[1] >= next_ele[0]: #controllo che la fine di una non venga dopo l'inizio della successiva
                error_comment = 'Nel gene ' + gene + ' nel trascritto ' + transcript + ' l\'esone che inizia a ' + start_end[0] + ' e finisce a ' + start_end[1] + ' si sovrappone all\'esone successivo'
                for row in gtf_file_rows: #cerrco il record in cui è stato commesso l'errore
                    gene_row = re.findall(gene_regexpr, row.rstrip().split('\t')[8])[0]
                    transcript_row = re.findall(transcript_regexpr, row.rstrip().split('\t')[8])[0]
                    start_row = row.rstrip().split('\t')[3]
                    end_row = row.rstrip().split('\t')[4] 
                    feature = row.rstrip().split('\t')[2]
                    if gene_row == gene and transcript == transcript_row and start_end[0] == start_row and start_end[1] == end_row and feature == 'exon':
                        #una volta trovato il record stammpo l'errore
                        n_row = gtf_file_rows.index(row)
                        write_error(report_file_name, error_comment, str(n_row))
                        sys.exit()

write_report(report_file_name, 'Nessun esone di nessun trascritto si sovrappone ad un altro per ogni gene nel file ')
                
        

SystemExit: 

**Per ogni Trascritto controllo che gli indici di inizio e di fine dei segmenti della CDS non si sovappongano** 

In caso contrario stampo l'errore e fermo l'esecuzione.

Costruisco un dizionario che Faccia corrispondere ad ogni gene i suoi trascritti

In [185]:
#stessa proceedura del punto di prima, solo per i record CDS
id_dict = dict()

for row in gtf_file_rows:
    if row.rstrip().split('\t')[2] == 'CDS':
        hugo_name = re.findall(gene_regexpr, row.rstrip().split('\t')[8])[0]
        transcript_id = re.findall(transcript_regexpr, row.rstrip().split('\t')[8])[0]
    
        transcript_set = id_dict.get(hugo_name, set())
        transcript_set.add(transcript_id)
        id_dict.update([(hugo_name, transcript_set)])

Costruisco un dizionario che faccia corrispondere ad ogni trascritto, una lista di tuple con indice di inizio e fine

In [186]:
transcript_dict = dict()

for row in gtf_file_rows:
    if row.rstrip().split('\t')[2] == 'CDS':
        transcript_id = re.findall(transcript_regexpr, row.rstrip().split('\t')[8])[0]
        start = row.rstrip().split('\t')[3]
        end = row.rstrip().split('\t')[4]
    
        start_end_list = transcript_dict.get(transcript_id, list())
        start_end_list.append((start, end))
        transcript_dict.update([(transcript_id, start_end_list)])

Controllo che le tuple non si sovrappongano

In [187]:
for gene in id_dict:
    for transcript in id_dict[gene]:
        transcript_dict[transcript].sort() 
        for start_end in transcript_dict[transcript][:-1]:
            next_index = transcript_dict[transcript].index(start_end) + 1
            next_ele = transcript_dict[transcript][next_index]
            if start_end[1] >= next_ele[0]:
                error_comment = 'Nel gene ' + gene + ' nel trascritto ' + transcript + ' Il segmento di CDS che inizia a ' + start_end[0] + ' e finisce a ' + start_end[1] + ' si sovrappone al segmento successivo'
                print(error_comment)
                for row in gtf_file_rows:
                    gene_row = re.findall(gene_regexpr, row.rstrip().split('\t')[8])[0]
                    transcript_row = re.findall(transcript_regexpr, row.rstrip().split('\t')[8])[0]
                    start_row = row.rstrip().split('\t')[3]
                    end_row = row.rstrip().split('\t')[4]
                    feature = row.rstrip().split('\t')[2]
                    if gene_row == gene and transcript == transcript_row and start_end[0] == start_row and start_end[1] == end_row and feature == 'CDS':
                        n_row = gtf_file_rows.index(row)
                        write_error(report_file_name, error_comment, str(n_row))
                        sys.exit()

write_report(report_file_name, 'Nessun segmento della CDS di nessun trascritto si sovrappone ad un altro per ogni gene nel file')

**Controllo validità delle feature start_codon e stop_codon nel caso ci siano:**

Controllo che siano alpiù lunghe 3

In [189]:
gene_transcript_set_start = set() #Lo uso per memorizzare gene_id e transcript_id dei campi <start_codon> supposti non atomici
gene_transcript_set_stop = set() #Lo uso per memorizzare gene_id e transcript_id dei campi <stop_codon> supposti non atomici
for row in gtf_file_rows:
    feature  = row.rstrip().split('\t')[2]
    if feature == 'start_codon' or feature == 'stop_codon':
        start = int(row.rstrip().split('\t')[3])
        end = int(row.rstrip().split('\t')[4])
        length = end - start + 1
        if length > 3: #errore
            error_comment = 'La lunghezza della feature ' + feature + ' è ' + str(length) + '. Dovrebbe essere al massimo 3'
            n_row = gtf_file_rows.index(row)
            write_error(report_file_name, error_comment, str(n_row))
            sys.exit()
        elif length == 3: #verifico che <frame> sia 0
            frame = row.rstrip().split('\t')[7]
            if frame != '0': #per i record start_codon/stop_codon lunghi 3bp il frame deve essere = 0
                #errore
                error_comment = 'Il valore del campo <frame> è ' + frame + '. Dovrebbe essere 0 per feaure atomiche di tipo ' + feature
                n_row = gtf_file_rows.index(row)
                write_error(report_file_name, error_comment, str(n_row))
                sys.exit()
            #se non ho un errore, aggiungo lo hugo_name e il transcript_id al relativo set
            hugo_name = re.findall(gene_regexpr, row.rstrip().split('\t')[8])[0]
            transcript_id = re.findall(transcript_regexpr, row.rstrip().split('\t')[8])[0]
            if feature == 'start_codon':
                gene_transcript_set_start.add((hugo_name, transcript_id))
            else:
                gene_transcript_set_stop.add((hugo_name, transcript_id))
        else: #suppongo campo non atomico 
            hugo_name = re.findall(gene_regexpr, row.rstrip().split('\t')[8])[0]
            transcript_id = re.findall(transcript_regexpr, row.rstrip().split('\t')[8])[0]
            if feature == 'start_codon':
                gene_transcript_set_start.add((hugo_name, transcript_id))
            else:
                gene_transcript_set_stop.add((hugo_name, transcript_id))

Per start_codon e stop_codon controllo che la loro lunghezza sia 3

In [190]:
#per le due feaure inizializzo un dizionario che perogni gene e per ogni trascritto relativo a quel gene 
#faccia corrispondere la lunghezza sei campi start_codon/stop_codon
feature_length_start = dict.fromkeys(gene_transcript_set_start, 0)
feature_length_stop = dict.fromkeys(gene_transcript_set_stop, 0)

for row in gtf_file_rows:
    feature  = row.rstrip().split('\t')[2]
    hugo_name = re.findall(gene_regexpr, row.rstrip().split('\t')[8])[0]
    transcript_id = re.findall(transcript_regexpr, row.rstrip().split('\t')[8])[0]
    key = (hugo_name, transcript_id)
    if feature == 'start_codon' and key in gene_transcript_set_start:
        start = int(row.rstrip().split('\t')[3])
        end = int(row.rstrip().split('\t')[4])
        length = end - start + 1
        feature_length_start[key] += length
    if feature == 'stop_codon' and key in gene_transcript_set_stop:
        start = int(row.rstrip().split('\t')[3])
        end = int(row.rstrip().split('\t')[4])
        length = end- start + 1
        feature_length_stop[key] += length
#costruisco due liste per le due feature che contengano le coppie (gene_id, transcript_id) che non rispettano il vincolo
error_gene_transcript_start =[k for k, v in feature_length_start.items() if v != 3]
error_gene_transcript_stop = [k for k, v in feature_length_stop.items() if v != 3]
#se sono diverse dalla lista vuota ho un errore
if error_gene_transcript_start != []:
    error_comment = 'Per i seguenti coppie (gene_id, transcript_id) : ' + str(error_gene_transcript_start) + ' non è stata rispettata la lunghezza della feature <start_codon>'
    write_error(report_file_name, error_comment, 'undefined') #non avrebbe senso specificare un linea per questo errore
    sys.exit()

if error_gene_transcript_stop != []:
    error_comment = 'Per i seguenti coppie (gene_id, transcript_id) : ' + str(error_gene_transcript_stop) + ' non è stata rispettata la lunghezza della feature <stop_codon> '
    write_error(report_file_name, error_comment, 'undefined')
    sys.exit()

write_report(report_file_name, 'Nel caso siano presenti start_codon e stop_codon, la lunghezza di queste feature è rispettata')